In [72]:
import pandas as pd
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
# from simpletransformers.ner import NERModel, NERArgs


In [73]:
# read into dataframe
path = "/Users/cyrus/Desktop/2023 eBay ML Challenge Data/Train_Tagged_Titles.tsv"
df = pd.read_csv(path, sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
df["Record Number"] = df["Record Number"].astype(int)
# train_df = df[df["labels"]!="np.NaN"]
df

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
55178,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sportschuhe,Produktart
55179,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Wanderschuh,NaN
55180,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Big,No Tag
55181,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Size,No Tag


In [45]:
# # combine NaN tokens ("New", "Balance" --> "New Balance")
# prev_val = None
# max = len(df[df["Record Number"]<=5000])
# for i in reversed(range(0,max)):
#     dftemp = df.iloc[i]
#     token = dftemp.iat[2]
#     tag = dftemp.iat[3]

#     # check if tag is None
#     if tag is np.NaN:
#         if prev_val is None:
#             prev_val = []
#         prev_val.insert(0, token)
#         df.iat[i,3] = "drop"
#     else:
#         if prev_val is not None:
#             prev_val.insert(0, token)
#             df.iat[i, 2] = " ".join(prev_val)
#             prev_val = None

# df = df[df["Tag"] != "drop"]
# df

In [74]:
# forward fill NaN values with last valid value
df = df.ffill()
# combine each sentence_id
title = list(df.groupby("Record Number")['Title'].agg('first'))
tags = list(df.groupby("Record Number")['Tag'].apply(list))
train_df = pd.DataFrame({'Title':title, 'Tags':tags})
train_df

,Title,Tags
0,Supreme Nike SB Dunk High By any Means Red US1...,"[Modell, Marke, Produktlinie, Produktlinie, Sc..."
1,New Balance 530 Männer und Frauen Laufschuhe m...,"[Marke, Marke, Modell, Abteilung, No Tag, Abte..."
2,♥ MICHAEL KORS Sneaker Gr 39 ♥,"[No Tag, Marke, Marke, Stil, No Tag, EU-Schuhg..."
3,New Balance ML 574 EGO Turnschuhe grün,"[Marke, Marke, Modell, Modell, Modell, Produkt..."
4,"Nike air jordan 9 og , space jam , hare , bugs...","[Marke, Produktlinie, Produktlinie, Modell, Mo..."
...,...,...
4995,Adidas Raf Simons Sneakers,"[Marke, Produktlinie, Produktlinie, Stil]"
4996,"Ludwig Reiter Andalusier , Nubuk navy Schuhe D...","[Marke, Marke, Modell, No Tag, Obermaterial, F..."
4997,Adidas Superstar Laceless / Adidas Originals /...,"[Marke, Produktlinie, Verschluss, No Tag, Mark..."
4998,HUGO BOSS Herren Schuhe Schnürschuhe Sneaker V...,"[Marke, Marke, Abteilung, Produktart, Produkta..."


In [42]:
# df = df.rename(columns={"Title": "title", "Record Number":"sentence_id","Token":"words","Tag":"labels"})
# df = df[["title","words","labels"]]
# df

In [22]:
X = df[["title", "words"]]
Y = df["labels"]

In [78]:
labels = df["Tag"].unique()

36

In [91]:
lengths = train_df['Tags'].apply(len)
lengths.max()

28

In [8]:
train_data = pd.DataFrame({"sentence_id":X["sentence_id"], "words":X["words"], "labels":Y})
# train_data

In [9]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 0.001
args.overwrite_output_dir = True
args.train_batch_size = 100
args.eval_batch_size = 100

NameError: name 'NERArgs' is not defined

In [ ]:
model = NERModel('bert', 'dbmdz/bert-base-german-uncased',labels=labels,args=args, use_cuda=False)

Some weights of the model checkpoint at dbmdz/bert-base-german-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint a

In [ ]:
model.train_model(train_data, eval_data=df, acc=accuracy_score)

/Users/cyrus/miniforge3/envs/moon/lib/python3.10/site-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/50 [00:00<?, ?it/s]

(50, 2.7124241256713866)

In [ ]:
model.model.save_pretrained('bert_model')
model.tokenizer.save_pretrained('bert_model')
model.config.save_pretrained('bert_model')

In [ ]:
result, model_outputs, preds_list = model.eval_model(df)

/Users/cyrus/miniforge3/envs/moon/lib/python3.10/site-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/10 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 2.5846244884490965,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

In [ ]:
prediction, model_output = model.predict(["Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'Supreme': 'No Tag'},
  {'Nike': 'No Tag'},
  {'SB': 'No Tag'},
  {'Dunk': 'No Tag'},
  {'High': 'No Tag'},
  {'By': 'No Tag'},
  {'any': 'No Tag'},
  {'Means': 'No Tag'},
  {'Red': 'No Tag'},
  {'US10': 'No Tag'},
  {'EU44': 'No Tag'},
  {'Supreme': 'No Tag'},
  {'Box': 'No Tag'},
  {'Logo': 'No Tag'},
  {'Air': 'No Tag'},
  {'Force': 'No Tag'}]]

In [ ]:
prediction, model_output = model.predict(["♥ MICHAEL KORS Sneaker Gr 39 ♥"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'♥': 'No Tag'},
  {'MICHAEL': 'No Tag'},
  {'KORS': 'No Tag'},
  {'Sneaker': 'No Tag'},
  {'Gr': 'No Tag'},
  {'39': 'No Tag'},
  {'♥': 'No Tag'}]]